In [ ]:
# Import all lib and modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
url = "https://www.kaggle.com/code/juanzi/house-price-prediction"

In [75]:
df = pd.read_csv(r'C:\Users\Hanif ullah laptop\Desktop\House price Prediction\data\Clean\preprocessed_data.csv')
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,year_sold,month_sold
0,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,98133,2014,5
1,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,98119,2014,5
2,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,98042,2014,5
3,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,98008,2014,5
4,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,98052,2014,5


In [76]:
#  train test split
from sklearn.model_selection import train_test_split
X = df.drop('price', axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [77]:
# # column transformer and model building
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression
# from sklearn.pipeline import Pipeline
# # winsorizer
# from feature_engine.outliers import Winsorizer
# # selectkbest
# from sklearn.feature_selection import SelectKBest, f_regression
# # create column transformer
# column_transformer = ColumnTransformer(transformers=[
#     ('winsorizer', Winsorizer(capping_method='iqr', tail='both', fold=1.5), df.select_dtypes(include=['int64', 'float64']).columns), 
#     ('scaler', StandardScaler(), df.columns.tolist()),
#     ('selector', SelectKBest(score_func=f_regression, k=10), X.columns)
# ], remainder='passthrough')

In [78]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from feature_engine.outliers import Winsorizer
from xgboost import XGBRegressor
# select numeric columns
cont_cols = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','sqft_above','sqft_basement']
cat_cols = ['waterfront','view','condition','zipcode','yr_built','yr_renovated','year_sold','month_sold']

preprocessor = ColumnTransformer(
    transformers=[
        ('winsorizer', Winsorizer(capping_method='iqr', tail='both', fold=1.5), cont_cols),
        ('scaler', StandardScaler(), cont_cols)
     
    ],
    remainder='passthrough'
)


# define full pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selector', SelectKBest(score_func=f_regression, k=10)),
    # ('selector', SelectKBest(score_func=f_regression, k=10), cont_cols + cat_cols),
    ('model', LinearRegression())
])


In [79]:
# pipeline
# Xgboost import 
# from xgboost import XGBRegressor
# pipeline = Pipeline(steps=[
#     ('preprocessor', column_transformer),
#     ('regressor', XGBRegressor())
# ])

In [80]:
#  score the model
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f'MAE: {mae}')    
print(f'RMSE: {rmse}')
print(f'R2: {r2}')


MAE: 221748.68263798227
RMSE: 1000775.7790232905
R2: 0.01793817312294399
